**Questão 1:** Modele este problema como um problema de otimização, identificando:
- as variáveis de decisão;
- a função objetivo;
- as restrições envolvidas, incluindo as de integralidade, se houver.

**Resposta:** 
- **Variáveis de decisão:** $x_i$, que é o número de cortes de tamanho $i$. 
- **Função objetivo:** variaveis de decisão ponderando o lucro do corte. (maximizar lucro total)
- **Restrições envolvidas:** Temos que garantir que as variáveis sejam inteiras, positivas e que o
conjunto dos cortes seja factível (todos os cortes devem somar a ser o comprimento da barra)

$$
\begin{align*}
& \max_{x_i} & &  \sum_{i \in \{1\dots n\}} x_i\times l(i)  \nonumber \\
& \textrm{s.t.} & & \sum_{i \in \{1\dots n\}} i \times x_{i} = n,\\
& & &  x_{i} \geq 0\nonumber, \\
& & &  x_{i} \in \mathbb{N} \nonumber. \\
\end{align*} $$

**Questão 2:** Em uma estratégia de busca por soluções de forma enumerativa, podemos enumerar os padrões de corte e
avaliar o seu lucro obtido. Pensando que uma barra de tamanho n pode (ou não) ser cortada em n − 1 posições
distintas (cada uma espaçada de 1 unidade de medida), quantas soluções devem ser testadas (considere repetições)?

**Resposta:** Sabendo que temos n-1 escolha binárias para a barra (cortar ou não cortar) o número de casos que precisamos testar é $2^{(n-1)}$ possíveis soluções (complexidade exponencial). 

**Questão 3:** Projete, por indução, um algoritmo recursivo que encontre a solução ótima para este problema. Para tanto,
as seguintes perguntas devem ser respondidas pelo seu projeto:
- Qual é o caso base para a recursão?
- Como construir a solução ótima para a barra de tamanho k se conhecermos as soluções ótimas para as barras
de tamanho 1, · · · , k − 1.
- Ao final da execução, como reconstruir a solução ótima? Lembre de memorizar a melhor escolha a cada passo!

**Resposta:**

 A ideia é a seguinte: Seja $T(k) \in \mathbb{N}^k$ a solução do nosso problema para a barra de tamanho k, onde $T_i(k)$ representa o número de cortes de tamanho $i$. 

Para o caso base podemos tomar que em uma barra de tamanho 0 a solução é não fazer nenhum corte, então $T(0) = [0, 0 \dots 0] $.

Se soubermos a solução para $\{1, 2, 3 \dots k-1\}$, para montar a solução ótima podemos fazer da seguinte maneira recursiva $$T(k) = \argmax_{i \in \{1, 2 \dots k-1\}} \left(l(i) + \sum_{j \in \{1, 2, \dots k\}}l(j) \times T_j(k-i)\right)$$

(Ps: eu tentei usar notação de vetor e ficou meio feio, né?)

Assim para encontrar o valor máximo do total de lucro podemos fazer: 
$$L_\text{max} = \sum_{i \in \{1, 2, \dots k\}}l(i)\times T_i(k)$$

(Ps: No código prefiri implementar com dicts ao invés de listas, mas ainda é a mesma ideia)

In [26]:
def bar_cut_problem_recursive(n, l_dict):
    max_value = 0 
    solution_assignments = {i: 0 for i in range(1,n+1)}
    chosen_assignments = {}
    chosen_cut = 0

    if n == 0: 
        return 0, {}
    
    for idx in range(1,n+1):
        current_value, previous_assigments = bar_cut_problem_recursive(n-idx, l_dict)
        current_value = current_value + l_dict[idx]

        if current_value > max_value: 
            max_value = current_value
            chosen_cut = idx
            chosen_assignments = previous_assigments

    for key in chosen_assignments.keys():
        solution_assignments[key] = chosen_assignments[key]

    solution_assignments[chosen_cut] = solution_assignments[chosen_cut] + 1

    return max_value, solution_assignments

# Exemplo de uso
n = 4
l_dict = {1: 4, 2: 8, 3: 13, 4: 15}

max_value, x_opt = bar_cut_problem_recursive(n, l_dict)

print("Atribuição ótima de Xi:")
for i in sorted(x_opt.keys()):
    print(f"x_{i} = {x_opt[i]}")

print(f"Lucro Máximo: {max_value}")

Atribuição ótima de Xi:
x_1 = 1
x_2 = 0
x_3 = 1
x_4 = 0
Lucro Máximo: 17


**Questão 4:** Um problema que pode acontecer em algoritmos recursivos como o projetado acima é o re-cálculo de
soluções de subproblemas durante as chamadas recursivas. Isto é, por exemplo, para resolver o problema com
n = 4, o problema com n = 2 é resolvido em mais de uma chamada recursiva. Isso acontece com o seu algoritmo?
Se isto acontece, o seu algoritmo pode facilmente se aproximar da simples enumeração e testagem de soluções.

**Resposta:** Sim, isto acontece nesse algoritmo. 

**Questão 5:** Para evitar o recômputo de soluções já calculadas, proponha uma forma ordenada de resolver o problema,
partindo de cortes menores e construindo a solução ótima para problemas maiores. Use esta forma de resolução
para remover as chamadas recursivas, reescrevendo-o usando laços (loops). Quantas operações são necessárias, nesta
versão, para resolver o problema?

**Resposta:** 



In [119]:
import numpy as np
def bar_cut_with_loop(n, l_dict):
    dp = np.zeros([n+1, n])
    l_array = np.array(list(l_dict.values()))
    
    for i in range(n + 1):
        chosen_idx = -1
        max_profit = 0
        for j in range(1,i+1):
            profit_sum = l_array[j-1] + np.dot(dp[i-j,:], l_array)
            if profit_sum > max_profit:
                chosen_idx = j-1
                max_profit = profit_sum
            
        if chosen_idx >= 0:
            dp[i,:] = dp[i-(chosen_idx+1),:] 
            dp[i, chosen_idx] = dp[i, chosen_idx] + 1
        #print(f"Iteração para barra de tamanho {i}")
        #print(dp)

    max_value = np.dot(dp[n,:], l_array)

    return max_value, dp[n]

# Exemplo de uso
n = 4
l_dict = {1: 4, 2: 8, 3: 13, 4: 15}

max_value, x_opt = bar_cut_with_loop(n, l_dict)

print("Atribuição ótima de Xi:")
for i, value in enumerate(x_opt):
    print(f"x_{i+1} = {int(value)}")

print(f"Lucro Máximo: {int(max_value)}")

Atribuição ótima de Xi:
x_1 = 1
x_2 = 0
x_3 = 1
x_4 = 0
Lucro Máximo: 17


**Questão 6:** Use o algoritmo projetado para resolver o caso de teste fornecido no moodle.